In [8]:
import os
from llama_index.llms.perplexity import Perplexity
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

pplx_api_key = os.getenv("PERPLEXITY_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')

query_llm = Perplexity(
    api_key=pplx_api_key, model="sonar-small-online", temperature=0.5
)

entity_llm = OpenAI(temperature=0.2, model="gpt-4")

In [9]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext

username = "neo4j"
password = os.getenv('NEO4J_PW')
url = os.getenv('NEO4J_URL')
database = "neo4j"

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


In [3]:
from llama_index.core.indices.knowledge_graph.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from entity_extraction.entity_extraction import custom_entity_extract_fn
from synonym_expand.synonym import custom_synonym_expand_fn

graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    llm=entity_llm,
    # entity_extract_fn=custom_entity_extract_fn,
    retriever_mode="keyword",
    with_nl2graphquery=True,
    synonym_expand_fn=custom_synonym_expand_fn
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

In [16]:
from llama_index.core.llms import ChatMessage

query = "tell me about crypto"

pre_response = query_engine_with_nl2graphquery.query(
    query,
)

if pre_response.metadata is None:
    # 1) answer needs to be queried from Llama2 endpoint
    messages_dict = [
        {"role": "system", "content": "Be precise and concise."},
        {"role": "user", "content": query},
    ]
    messages = [ChatMessage(**msg) for msg in messages_dict]

    external_response = query_llm.chat(messages)
    print(external_response)

    # 2) answer needs to be stored into txt file for loading into KG
    with open('data/external_response.txt', 'w') as f:
        print(external_response, file=f)
else:
    print("From Graph: " + str(pre_response))
    print(list(list(pre_response.metadata.values())[0]["kg_rel_map"].keys()))

assistant: Crypto refers to a digital currency designed to function as a medium of exchange through a decentralized computer network, independent of a central authority like a government or bank. It utilizes cryptography to secure transactions and maintain the integrity of the currency's supply and ownership records. Cryptocurrencies are stored in digital wallets and operate on a decentralized system known as blockchain, which is a public ledger recording all transactions. The first cryptocurrency was Bitcoin, introduced in 2009, and there are now over 25,000 other cryptocurrencies available, with varying levels of adoption and regulation.


In [10]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=['data/external_response.txt']
).load_data()

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=5,
)

In [18]:
post_response = query_engine_with_nl2graphquery.query(
   query,
)

print(post_response)
print(list(list(post_response.metadata.values())[0]["kg_rel_map"].keys()))

Cryptocurrency is a digital currency that utilizes cryptography to secure transactions and maintain the integrity of its supply. It is designed to function as a medium of exchange and refers to digital currency.
['Crypto']
